In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense, MaxPool2D, Dropout, Flatten, Conv2D
from keras.models import Sequential
from keras.preprocessing import image 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
from glob import glob
from sklearn.model_selection import train_test_split

In [2]:
dataset_path = 'D:\\dataset\\brain'

# Data generators


In [3]:

train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, zoom_range=0.2, shear_range=0.2,validation_split=0.2)


# Load Dataset

In [9]:
train_data = train_datagen.flow_from_directory(
        dataset_path,
        target_size=(256,256),
        batch_size=10,
        class_mode='binary',subset='training')

val_data = train_datagen.flow_from_directory(
        dataset_path,
        target_size=(256,256),
        batch_size=10,
        class_mode='binary',subset='validation')

test_data = train_datagen.flow_from_directory(
        dataset_path,
        target_size=(256,256),
        batch_size=10,
        class_mode='binary',
        shuffle=False,
        subset='validation')



Found 203 images belonging to 2 classes.
Found 50 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


# CNN model

In [10]:
model = Sequential()

model.add(Conv2D (filters= 16 , kernel_size= (3,3), activation= 'relu', input_shape=(256,256,3)))

model.add(Conv2D(filters= 36, kernel_size= (3,3), activation= 'relu' ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters= 64, kernel_size= (3,3), activation= 'relu')) 
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters= 128, kernel_size= (3,3), activation= 'relu' )) 
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout (rate= 0.25))

model.add(Flatten())
model.add(Dense(units=64, activation= 'relu'))
model.add(Dropout(rate= 0.25 ))

model.add(Dense (units= 1, activation= 'sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 254, 254, 16)      448       
                                                                 
 conv2d_5 (Conv2D)           (None, 252, 252, 36)      5220      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 126, 126, 36)     0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 124, 124, 64)      20800     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 60, 60, 128)      

In [11]:
model.compile(optimizer='adam',loss=keras.losses.binary_crossentropy, metrics=["accuracy"])

# Early stopping & Model check point


In [12]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

es= EarlyStopping(monitor="val_accuracy",min_delta=0.01,patience=3,verbose=1,mode="auto")

mc= ModelCheckpoint(monitor="val_accuracy",filepath="D:\\bestmodel.h5",verbose=1,save_best_only=True,mode="auto")

call=[es,mc]

# Model Training

In [13]:
hs=model.fit_generator(generator=train_data, steps_per_epoch=5,epochs=10, 
                       verbose=1, validation_data=val_data, validation_steps=5,
                       callbacks=call)

C:\Users\Sid\AppData\Local\Temp\ipykernel_988\4192182303.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hs=model.fit_generator(generator=train_data, steps_per_epoch=5,epochs=10,


Epoch 1/10
5/5 [==============================] - ETA: 0s - loss: 0.5754 - accuracy: 0.7209
Epoch 1: val_accuracy improved from -inf to 0.46000, saving model to D:\bestmodel.h5
5/5 [==============================] - 18s 3s/step - loss: 0.5754 - accuracy: 0.7209 - val_loss: 1.6808 - val_accuracy: 0.4600
Epoch 2/10
5/5 [==============================] - ETA: 0s - loss: 0.9130 - accuracy: 0.6600
Epoch 2: val_accuracy improved from 0.46000 to 0.62000, saving model to D:\bestmodel.h5
5/5 [==============================] - 18s 4s/step - loss: 0.9130 - accuracy: 0.6600 - val_loss: 0.7417 - val_accuracy: 0.6200
Epoch 3/10
5/5 [==============================] - ETA: 0s - loss: 0.6563 - accuracy: 0.6000
Epoch 3: val_accuracy improved from 0.62000 to 0.70000, saving model to D:\bestmodel.h5
5/5 [==============================] - 16s 3s/step - loss: 0.6563 - accuracy: 0.6000 - val_loss: 0.6233 - val_accuracy: 0.7000
Epoch 4/10
5/5 [==============================] - ETA: 0s - loss: 0.5945 - accurac

In [14]:
h=hs.history
h.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

# Model Accuracy


In [15]:
from keras.models import load_model, Model

model=load_model("D:\\bestmodel.h5")

In [17]:
acc= model.evaluate_generator(test_data)[1]
print(f"accurary of model {acc}")

C:\Users\Sid\AppData\Local\Temp\ipykernel_988\342094648.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc= model.evaluate_generator(test_data)[1]


accurary of model 0.7599999904632568


In [18]:
from keras.applications.mobilenet import MobileNet

In [26]:
base_model= MobileNet(input_shape=(256,256,3),include_top=False)

In [27]:
for layer in base_model.layers:
    layer_trainable = False

In [28]:
X= Flatten()(base_model.output)
X=Dense(units=1, activation='sigmoid')(X)

model= Model(base_model.input,X)

In [29]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 128, 128, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 128, 128, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 128, 128, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 128, 128, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 128, 128, 32)     128       
 ation)                                                    

In [30]:
model.compile(optimizer='rmsprop',loss=keras.losses.binary_crossentropy, metrics=["accuracy"])

In [31]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

es= EarlyStopping(monitor='val_accuracy',min_delta=0.01,patience=3,verbose=1)

mc= ModelCheckpoint(monitor='val_accuracy',filepath="bestmodel.h5",verbose=1,save_best_only=True)

cb=[es,mc]

In [32]:
hist=model.fit_generator(generator=train_data, steps_per_epoch=5,epochs=10, 
                       validation_data=val_data, validation_steps=5,
                       callbacks=cb)

C:\Users\Sid\AppData\Local\Temp\ipykernel_988\2245452028.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist=model.fit_generator(generator=train_data, steps_per_epoch=5,epochs=10,


Epoch 1/10
5/5 [==============================] - ETA: 0s - loss: 22.9098 - accuracy: 0.5000
Epoch 1: val_accuracy improved from -inf to 0.62000, saving model to bestmodel.h5
5/5 [==============================] - 36s 5s/step - loss: 22.9098 - accuracy: 0.5000 - val_loss: 9.7805 - val_accuracy: 0.6200
Epoch 2/10
5/5 [==============================] - ETA: 0s - loss: 13.5685 - accuracy: 0.5800
Epoch 2: val_accuracy did not improve from 0.62000
5/5 [==============================] - 20s 4s/step - loss: 13.5685 - accuracy: 0.5800 - val_loss: 15.5841 - val_accuracy: 0.6200
Epoch 3/10
5/5 [==============================] - ETA: 0s - loss: 6.0309 - accuracy: 0.6977
Epoch 3: val_accuracy did not improve from 0.62000
5/5 [==============================] - 19s 4s/step - loss: 6.0309 - accuracy: 0.6977 - val_loss: 15.8085 - val_accuracy: 0.3800
Epoch 4/10
5/5 [==============================] - ETA: 0s - loss: 5.7481 - accuracy: 0.7600
Epoch 4: val_accuracy improved from 0.62000 to 0.74000, savin

In [33]:
model=load_model("D:\\bestmodel.h5")

In [34]:
acc= model.evaluate_generator(test_data)[1]
print(f"accurary of model {acc}")

C:\Users\Sid\AppData\Local\Temp\ipykernel_988\342094648.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc= model.evaluate_generator(test_data)[1]


accurary of model 0.7799999713897705
